In [10]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [12]:
raw_data = pd.read_csv(os.path.join("Resources", "Output", "HoneybeeDatasetSVCModel.csv"))
raw_data

,Region,FIPS,ColonyCount,ColonyGrowth,GrowthOutcome,TotalProduction,YieldPerColony,PricePerLB,ProductionValue,Stocks,YearRecorded,ClothianidinLB,ImidaclopridLB,ThiamethoxamLB,AcetamipridLB,ThiaclopridLB,CombinedNeonicLB
0,3,1,16000,NaN,NaN,928000,58,0.69,640000,28000,1995,0.00,1579.61,0.00,0.0,0.0,1579.61
1,3,1,15000,-6.25,0.0,960000,64,0.87,835000,96000,1996,0.00,819.24,0.00,0.0,0.0,819.24
2,3,1,14000,-6.67,0.0,924000,66,0.81,748000,92000,1997,0.00,14781.55,0.00,0.0,0.0,14781.55
3,3,1,16000,14.29,1.0,1136000,71,0.72,818000,159000,1998,0.00,4048.35,0.00,0.0,0.0,4048.35
4,3,1,17000,6.25,1.0,1156000,68,0.56,647000,185000,1999,0.00,2758.42,0.00,0.0,0.0,2758.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,4,56,50000,42.86,1.0,2550000,51,1.87,4769000,459000,2012,2590.21,610.68,933.22,0.0,0.0,4134.11
891,4,56,47000,-6.00,0.0,3102000,66,2.11,6545000,558000,2013,1853.87,343.70,1160.07,0.0,0.0,3357.64
892,4,56,38000,-19.15,0.0,2318000,61,2.08,4821000,255000,2014,2782.23,569.23,1248.04,0.0,0.0,4599.50
893,4,56,38000,0.00,0.0,2926000,77,1.90,5559000,146000,2015,0.00,252.21,41.89,0.0,0.0,294.10


In [13]:
honeybee_data = raw_data.apply(pd.to_numeric, errors='coerce')
honeybee_data = honeybee_data.dropna()
honeybee_data.head()

,Region,FIPS,ColonyCount,ColonyGrowth,GrowthOutcome,TotalProduction,YieldPerColony,PricePerLB,ProductionValue,Stocks,YearRecorded,ClothianidinLB,ImidaclopridLB,ThiamethoxamLB,AcetamipridLB,ThiaclopridLB,CombinedNeonicLB
1,3,1,15000,-6.25,0.0,960000,64,0.87,835000,96000,1996,0.0,819.24,0.0,0.0,0.0,819.24
2,3,1,14000,-6.67,0.0,924000,66,0.81,748000,92000,1997,0.0,14781.55,0.0,0.0,0.0,14781.55
3,3,1,16000,14.29,1.0,1136000,71,0.72,818000,159000,1998,0.0,4048.35,0.0,0.0,0.0,4048.35
4,3,1,17000,6.25,1.0,1156000,68,0.56,647000,185000,1999,0.0,2758.42,0.0,0.0,0.0,2758.42
5,3,1,16000,-5.88,0.0,1248000,78,0.59,736000,187000,2000,0.0,3305.17,0.0,0.0,0.0,3305.17


In [14]:
target = honeybee_data['GrowthOutcome']
target_names = ['Growth', 'No-Growth']

In [15]:
X = honeybee_data.drop(columns=['GrowthOutcome', 'ColonyGrowth', 'YearRecorded','FIPS','Region'])
y = honeybee_data["GrowthOutcome"]
feature_names = X.columns
print(X.shape, y.shape)

(894, 12) (894,)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train

,ColonyCount,TotalProduction,YieldPerColony,PricePerLB,ProductionValue,Stocks,ClothianidinLB,ImidaclopridLB,ThiamethoxamLB,AcetamipridLB,ThiaclopridLB,CombinedNeonicLB
36,20000,1040000,52,1.45,1508000,562000,586.43,20523.27,1159.85,2748.72,0.0,25018.28
47,52000,4628000,89,0.53,2453000,2592000,0.00,1526.04,0.00,0.00,0.0,1526.04
591,22000,1760000,80,0.85,1496000,827000,0.00,1171.98,0.00,0.00,0.0,1171.98
687,270000,17820000,66,1.42,25304000,6237000,56707.52,24792.30,66206.80,0.00,0.0,147706.63
751,26000,1300000,50,0.98,1274000,299000,106.48,27.12,111.99,42.77,0.0,288.36
...,...,...,...,...,...,...,...,...,...,...,...,...
716,6000,330000,55,4.88,1610000,69000,8762.27,7184.20,35602.67,1046.98,0.0,52596.12
768,7000,567000,81,0.92,522000,249000,0.00,153.88,0.00,0.00,0.0,153.88
73,480000,32160000,67,1.39,44702000,6432000,0.00,84903.77,10174.55,27101.43,0.0,122179.74
236,37000,2257000,61,2.17,4898000,1196000,370881.90,108750.29,130703.70,24.25,0.0,610360.13


In [17]:
X_test

,ColonyCount,TotalProduction,YieldPerColony,PricePerLB,ProductionValue,Stocks,ClothianidinLB,ImidaclopridLB,ThiamethoxamLB,AcetamipridLB,ThiaclopridLB,CombinedNeonicLB
832,6000,330000,55,1.14,376000,155000,0.00,266.10,0.00,0.00,0.00,266.10
875,38000,2356000,62,0.75,1767000,660000,0.00,115.30,0.00,0.00,0.00,115.30
796,6000,276000,46,2.43,671000,63000,6392.74,4167.62,5986.43,1566.16,56.88,18169.84
234,27000,1323000,49,1.92,2540000,463000,135174.89,166578.86,106157.43,12.79,0.00,407923.97
57,24000,1368000,57,1.42,1943000,301000,9583.27,22221.27,46708.24,769.63,0.00,79282.42
...,...,...,...,...,...,...,...,...,...,...,...,...
600,14000,784000,56,1.40,1098000,282000,21389.25,7916.14,16028.27,299.39,169.76,45802.80
665,9000,810000,90,1.26,1021000,65000,0.00,14.11,0.00,0.00,0.00,14.11
224,30000,2010000,67,0.68,1367000,1206000,0.00,135.36,0.00,0.00,0.00,135.36
287,44000,5060000,115,0.70,3542000,759000,0.00,1088.64,0.00,0.00,0.00,1088.64


In [18]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.5580357142857143

In [19]:
rf = RandomForestClassifier(n_estimators=800)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.6473214285714286

In [20]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.11735478516695808, 'ProductionValue'),
 (0.10599649793329417, 'TotalProduction'),
 (0.10041064459682175, 'ImidaclopridLB'),
 (0.10012143771445407, 'Stocks'),
 (0.09781443460285415, 'PricePerLB'),
 (0.09764032214792093, 'CombinedNeonicLB'),
 (0.09728977739573245, 'YieldPerColony'),
 (0.09028586885475282, 'ColonyCount'),
 (0.0678708344228087, 'ThiamethoxamLB'),
 (0.053198102201682855, 'ClothianidinLB'),
 (0.05140303696348446, 'AcetamipridLB'),
 (0.02061425799923551, 'ThiaclopridLB')]

In [21]:
from sklearn.metrics import classification_report
honeybee_prediction = rf.predict(X_test)
print(classification_report(y_test, honeybee_prediction))

              precision    recall  f1-score   support

         0.0       0.69      0.85      0.76       149
         1.0       0.45      0.25      0.32        75

    accuracy                           0.65       224
   macro avg       0.57      0.55      0.54       224
weighted avg       0.61      0.65      0.62       224

